# Solo Node Demonstration
This notebook spins up an in-process governance ledger node and performs a single AEIP handshake to submit an ITP artifact.

In [ ]:
from protocol.aeip_handshake import AEIPHandshake
from protocol.ledger_node import GovernanceLedgerNode
from govspine.common.crypto import temporal_seal, sha3_512_hex
from govspine.common.schema import load_schema, json_dumps
from govspine.common.artifacts import Artifact

handshake = AEIPHandshake(handshake_id="notebook-demo", governance_scope="demo")
for step, persona in zip(handshake.AEIP_STEPS if hasattr(handshake, 'AEIP_STEPS') else ['Intent','Justify','CounterSign','Commit','Update'], ['persona.A','persona.B','persona.A','persona.B','persona.A']):
    payload = {"message": f"{step} payload", "temporalSeal": temporal_seal()}
    handshake.record(step=step, persona_id=persona, payload=payload)

schema = load_schema('schemas/itp_schema.json')
artifact_payload = {
    'id': 'itp-notebook',
    'version': '1.0',
    'source': 'notebook',
    'personaId': 'persona.A',
    'timestamp': temporal_seal().split('::')[0],
    'hash': '',
    'dignityCompliance': True,
    'objective': 'demo',
    'constraints': ['demo'],
    'evaluationCriteria': ['demo']
}
artifact_payload['hash'] = sha3_512_hex(json_dumps(artifact_payload))
artifact = Artifact(schema=schema, payload=artifact_payload)
node = GovernanceLedgerNode()
result = node.submit_artifact(artifact_type='ITP', artifact_payload=artifact_payload, handshake=handshake.to_dict())
result